In [1]:
ban=0
batch_size=4

In [2]:
import torch

device0 = torch.device('cuda:5' if torch.cuda.is_available() else "cpu")
device1 = torch.device('cuda:5' if torch.cuda.is_available() else "cpu")

In [3]:
# 读数据
import pandas as pd
df_train=pd.read_excel("../data/df_train.xlsx",index_col=0)
df_test=pd.read_excel("../data/df_test.xlsx",index_col=0)

In [4]:
import ast
label18=[]
for i in range(len(df_train)):
    new_list=[]
    old_list=ast.literal_eval(df_train.loc[i]['label'])
    df_train.loc[i]['label']=old_list
    for j in old_list:
        if j==0:
            new_list+=[0,0,0]
        if j==1:
            new_list+=[0,0,1]
        if j==2:
            new_list+=[0,1,1]
        if j==3:
            new_list+=[1,1,1]
    label18.append(new_list)

df_train['label18']=label18

In [5]:
label18=[]
for i in range(len(df_test)):
    new_list=[]
    old_list=ast.literal_eval(df_test.loc[i]['label'])
    df_test.loc[i]['label']=old_list
    for j in old_list:
        if j==0:
            new_list+=[0,0,0]
        if j==1:
            new_list+=[0,0,1]
        if j==2:
            new_list+=[0,1,1]
        if j==3:
            new_list+=[1,1,1]
    label18.append(new_list)

df_test['label18']=label18

In [6]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
df_train_text=[df_train['text'][i] for i in range(len(df_train))]
df_test_text=[df_test['text'][i] for i in range(len(df_test))]


In [7]:
text2id_train = tokenizer(
        df_train_text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )
input_ids_train=text2id_train["input_ids"]
mask_train=text2id_train["attention_mask"]

text2id_test = tokenizer(
        df_test_text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )
input_ids_test=text2id_test["input_ids"]
mask_test=text2id_test["attention_mask"]


In [8]:
df_train['input_ids']=input_ids_train.tolist()
df_train['mask']=mask_train.tolist()

df_test['input_ids']=input_ids_test.tolist()
df_test['mask']=mask_test.tolist()

In [9]:
def get_att(input_ids_train_):
    index=0
    _115=[]
    for i in input_ids_train_:
        if i==115:
#             print(i)
            _115.append(index)
        index+=1
#     print('_115:',_115)
    ###########################################################
                
    if len(_115)==0:
        att=[]
        for i in input_ids_train_:
            if i in [101,102,0]:
                att.append(0)
            else:
                att.append(1)
        return att
    
    
    ###########################################################
    
    
    att=[]

    index=0
    max_=0
    for i in input_ids_train_:
        if i in [101,102,0]:
            att.append(-1)
            index+=1
            continue
        att_t=[]
        for j in _115:
            att_t.append(abs(index-j))

        att.append(min(att_t))
        if min(att_t)>max_:
            max_=min(att_t)
        index+=1
        
#     print('att:',att)
    
    ###########################################################
    att0=[]
    sum_=0
    num_=0
    for j in att:
        if j==-1:
            att0.append(0)
        else:
            att0.append(max_-j)
            sum_+=max_-j
            num_+=1
    att1=[]
    sum_=sum_/num_
    for j in att0:
        att1.append(j/sum_)
        
#     print('att1:',att1)
    return att1

In [10]:
input_ids_train_att=[get_att(i) for i in input_ids_train]
input_ids_test_att=[get_att(i) for i in input_ids_test]

In [11]:
df_train['att']=input_ids_train_att
df_test['att']=input_ids_test_att

In [12]:
from torch.utils.data import Dataset
class SentimentDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        text = self.dataset.loc[idx, "text"]
        label = self.dataset.loc[idx, "label"]
#         print(label)
        input_ids = self.dataset.loc[idx, "input_ids"]
        mask = self.dataset.loc[idx, "mask"]
        label18=self.dataset.loc[idx, "label18"]
        att=self.dataset.loc[idx, "att"]
        sample = {"text": text, "label": label,"label18": label18,"input_ids":input_ids,"mask":mask,'att':att}
        # print(sample)
        return sample

In [13]:
#按batch_size分
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import torch

train_loader = DataLoader(
    SentimentDataset(df_train), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0
)
print(train_loader)

test_loader = DataLoader(
    SentimentDataset(df_test), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)
print(test_loader)

In [14]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F


class fn_cls(nn.Module):
    def __init__(self,device):
        super(fn_cls, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-chinese")
        self.model.to(device)
        self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(768, 18)

    def forward(self, x, att, attention_mask=None):
        #att:batch size*100
        outputs = self.model(x, attention_mask=attention_mask)
        x = outputs[0]#batch size*100*768
        
        
        #100*768    and   100 ---->>>>   768
        all_e=[]
        for i in range(len(att)):
            bb=att[i]#100
            aa=x[i]#100*768
            cc=aa*bb.view(100,1)
            all_e.append(torch.sum(cc,0))#1*768
            
        x=torch.stack(all_e)#batch size*768
        
        x = self.dropout(x)
        x = self.l1(x)
        return x

In [15]:
sigmoid = nn.Sigmoid()

In [16]:
import torch

if ban==0:
    cls = fn_cls(device0)
    cls.train()
else:
    cls=torch.load("../data/cls"+str(ban)+".model",map_location=device0)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
def _18to6_(output,round_=0):#18tensor 变 6list
    output0=[]
    for j in output:#18
        i=0
        s=''
        list6=[]
        while(i<len(j)):
            ok=j[i]*1.2+j[i+1]*1.1+j[i+2]*1
            if round_==1:
                list6.append(ok.round().int().tolist())
            else:
                list6.append(ok.tolist())
            i+=3
        output0.append(list6)
    return output0

def get_loss_test(output,A,round_=0):#8*18
    output_r=torch.Tensor(_18to6_(output,round_=1)).to(device0)
    output=_18to6_(output,round_=round_)
    # print(output_r,A)
    cor_add=(output_r == A).sum().item()
    sum0=0
    for i in range(len(output)):
        for j in range(6):
            sum0+=(output[i][j]-A[i][j])*(output[i][j]-A[i][j])
    return sum0,cor_add

def test(device_test):
    cls.to(device_test)
    cls.eval()

    correct = 0
    total = 0
    loss_test=0
    for batch_idx,batch in enumerate(test_loader):
        with torch.no_grad():
#             print(batch['att'])
            # print(batch['label'],batch['label18'])
            label6=torch.stack(batch['label']).t().to(device_test).float()
            # label18=torch.stack(batch['label18']).t().to(device_test).float()

            input_ids=torch.stack(batch['input_ids']).t().to(device_test)
            mask=torch.stack(batch['mask']).t().to(device_test)
            att=torch.stack(batch['att']).t().to(device_test).float()
            
            output = cls(input_ids, att,attention_mask=mask)
            output=sigmoid(output)

            total += len(output)*6
            loss_add,cor_add = get_loss_test(output, label6 ,round_=1)
            loss_test+=loss_add
            correct+=cor_add

            tes_score=1/(1+(loss_test/total) ** 0.5)
            acc_score=100.*correct/total

        

        print('[{}/{} ({:.0f}%)]\t正确分类的样本数：{}，样本总数：{}，准确率：{:.2f}%，score：{}'.format(
                    batch_idx, len(test_loader),100.*batch_idx/len(test_loader), 
                    correct, total,acc_score,
                    tes_score
            ),end='\r')
    print('[{}/{} ({:.0f}%)]\t正确分类的样本数：{}，样本总数：{}，准确率：{:.2f}%，score：{}'.format(
                    batch_idx, len(test_loader),100.*batch_idx/len(test_loader), 
                    correct, total,acc_score,
                    tes_score
            ))
#     cls.to(device_train)
    return tes_score,acc_score


In [18]:
# test(device1)

In [20]:
weight=[]
import numpy as np
sumn=np.zeros(18)
for i in range(len(df_train)):
    print(i,end='\r')
    sumn+=np.array(df_train.loc[i]['label18'])
#     print(sumn,end='\r')

print(sumn.sum())
print(sumn.tolist())
    

24585.0
[408.0, 722.0, 1132.0, 221.0, 646.0, 1868.0, 280.0, 752.0, 1566.0, 544.0, 1589.0, 2994.0, 378.0, 1202.0, 2130.0, 932.0, 2638.0, 4583.0]


In [21]:
sumn=sumn.tolist()
weight=[max(sumn)/i for i in sumn]
print(weight)
weight=torch.Tensor(weight).to(device0)
print(weight)

[11.232843137254902, 6.347645429362881, 4.048586572438163, 20.737556561085974, 7.094427244582043, 2.4534261241970023, 16.367857142857144, 6.094414893617022, 2.926564495530013, 8.424632352941176, 2.8842039018250474, 1.5307281229124916, 12.124338624338625, 3.812811980033278, 2.151643192488263, 4.917381974248927, 1.7373009855951478, 1.0]
tensor([11.2328,  6.3476,  4.0486, 20.7376,  7.0944,  2.4534, 16.3679,  6.0944,
         2.9266,  8.4246,  2.8842,  1.5307, 12.1243,  3.8128,  2.1516,  4.9174,
         1.7373,  1.0000], device='cuda:5')


In [22]:
from torch import optim
#BCEWithLogitsLoss就是把Sigmoid-BCELoss合成一步
criterion = nn.BCELoss(weight=weight)
criterion_hui=nn.MSELoss()
optimizer = optim.Adam(cls.parameters(), lr=1e-4)

In [23]:
import time  # 引入time模块
 

import torch
import matplotlib.pyplot as plt

def train(device_train,device_test,epoch_be=0):
    epoch_num=4
    accumulation_steps=32
    
    train_score=[]
    test_score=[]
    test_acc=[]

    for epoch_idx in range(epoch_be,epoch_be+epoch_num):
        cls.to(device_train)
        print("_________________________________________________________________")
        print("_________________________________________________________________")
        print("_________________ epoch:"+str(epoch_idx)+" start _________________")
        print("_________________________________________________________________")
        print("_________________________________________________________________")
        score_list=[]
        loss_a=0
        total=0
        correct=0
        start = time.time()
        for batch_idx,batch in enumerate(train_loader):
            label6=torch.stack(batch['label']).t().to(device_train).float()
            label18=torch.stack(batch['label18']).t().to(device_train).float()

            #计算output
            input_ids=torch.stack(batch['input_ids']).t().to(device_train)
            mask=torch.stack(batch['mask']).t().to(device_train)
            att=torch.stack(batch['att']).t().to(device_test).float()
            
            output = cls(input_ids, att,attention_mask=mask)
            output=sigmoid(output)
            output6=torch.Tensor(_18to6_(output,round_=0)).to(device_train)


            # 梯度积累
            loss0 = criterion(output, label18)
            lossh=criterion_hui(output6,label6)

            loss=loss0+lossh
            loss = loss/accumulation_steps
            loss.backward()

            #计算score
            with torch.no_grad():
                total += len(output)*6
                loss_add,cor_add = get_loss_test(output, label6,round_=0)
                loss_a+=loss_add
                correct+=cor_add

                tr_score=1/(1+(loss_a/total) ** 0.5)
                acc_score=100.*correct/total


            if((batch_idx+1) % accumulation_steps) == 0:
                # 每 accumulation_steps 次更新一下网络中的参数
                optimizer.step()
                optimizer.zero_grad()

            if ((batch_idx+1) % accumulation_steps) == 1:
                score_list.append(tr_score)
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tscore:{:.6f}\t准确率：{:.2f}%'.format(
                    epoch_idx, batch_idx, len(train_loader), 
                    100.*batch_idx/len(train_loader), tr_score,acc_score
                ),end='\r')

        #每个epoch结束：
        print('labels:', label6)
        print('pred:', output6)
#         plt.plot([i for i in range(len(score_list))], score_list)
#         plt.show()

        print('Train Epoch: {}\tscore:{:.6f}\t准确率：{:.2f}%'.format(epoch_idx,tr_score,acc_score))
        t_score,t_acc=test(device_test)

        train_score.append(tr_score)
        test_score.append(t_score)
        test_acc.append(t_acc)

        #保存模型
        end = time.time()
        print('epoch:',str(epoch_idx)+'_执行时间: ',end - start)
        torch.save(cls,"../data/cls_"+str(end)+"_"+str(epoch_idx)+"_"+str(round(tr_score.tolist(),4))+"_"+str(round(t_score.tolist(),4))+".model")

        print("_________________________________________________________________")
        print("_________________________________________________________________")
        print("_________________ epoch:"+str(epoch_idx)+" end _________________")
        print("_________________________________________________________________")
        print("_________________________________________________________________")

#     plt.plot([i for i in range(len(train_score))], train_score)
#     plt.show()             

#     plt.plot([i for i in range(len(test_score))], test_score)
#     plt.show()

#     plt.plot([i for i in range(len(test_acc))], test_acc)
#     plt.show()
    
train(device0,device1,epoch_be=0)


_________________________________________________________________
_________________________________________________________________
_________________ epoch:0 start _________________
_________________________________________________________________
_________________________________________________________________
labels: tensor([[0., 0., 0., 1., 0., 0.],.636596	准确率：83.25%
        [0., 0., 0., 0., 0., 0.]], device='cuda:5')
pred: tensor([[8.5158e-02, 1.4223e-02, 3.9106e-02, 1.5378e-01, 1.5377e-02, 1.2957e-01],
        [1.1675e-02, 6.7749e-05, 1.4100e-01, 3.9017e-03, 4.6558e-01, 3.3947e-02]],
       device='cuda:5')
Train Epoch: 0	score:0.636609	准确率：83.25%
[1669/1670 (100%)]	正确分类的样本数：35495，样本总数：40080，准确率：88.56%，score：0.6941369771957397
epoch: 0_执行时间:  419.872460603714
_________________________________________________________________
_________________________________________________________________
_________________ epoch:0 end _________________
____________________________________________

In [31]:
end = time.time()
torch.save(cls,"../data/cls_"+str(end)+"_"+str(2)+"_"+str(0.6886)+"_"+str(0.6942)+".model")


In [61]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

def get_output(device_pre,model,text,f=0):
    model.to(device_pre)
    model.eval()
    text2id = tokenizer(
        text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )
#     print(text,text2id)
    input_ids=text2id["input_ids"].to(device_pre)
    mask=text2id["attention_mask"].to(device_pre)
    att=torch.tensor([get_att(input_ids[0])]).to(device_pre)
#         print(text2id)
    output = model(input_ids,att, attention_mask=mask)
    
    print(output)
    output=sigmoid(output)
    print(output)
    
#     output=sigmoid(output)*4
    if f==1:
        return _18to6_(output,round_=1)
    else:
        return _18to6_(output,round_=0)

In [62]:
text = ['*与n3：啊？']
# 爱、乐、惊、怒、恐、哀
print(get_output(device1,cls,text,1))

tensor([[-5.1787, -4.2691, -2.9211, -6.9531, -4.3885, -4.4666, -2.7147, -1.6942,
         -0.8299, -4.5495, -4.8081, -4.4052, -5.5299, -3.8132, -4.2456, -5.6641,
         -5.1396, -4.1083]], device='cuda:5', grad_fn=<AddmmBackward0>)
tensor([[0.0056, 0.0138, 0.0511, 0.0010, 0.0123, 0.0114, 0.0621, 0.1552, 0.3037,
         0.0105, 0.0081, 0.0121, 0.0040, 0.0216, 0.0141, 0.0035, 0.0058, 0.0162]],
       device='cuda:5', grad_fn=<SigmoidBackward0>)
[[0, 0, 1, 0, 0, 0]]
